In [1]:
case class Trade (
day: String,
trade_id: Int,
stock_symbol:String,
exchange:String,
trade_timestamp: String,
price: Float,
quantity: Int)

In [ ]:
val DateFormat = "([0-9]{4})-([0-9]{2})-([0-9]{2}) ([0-9]{2}):([0-9]{2}):([0-9]{2})".r 

In [ ]:
def dtt2dt(dt:String) = dt match {case DateFormat(y,mo,d,h,m,s) => s"$y-$mo-$d"}

In [2]:
val raw = sc.textFile(s"s3a://datastaxtraining/stockdata")

In [ ]:
raw.first

In [ ]:
val trades = raw
            .map{ _.split('|') 
               match { case Array(tid,tick,exch,dt,prc,qty)
                  => Trade(dtt2dt(dt),tid.toInt,tick,exch,dt,prc.toFloat,qty.toInt)}}

In [ ]:
trades.first

In [ ]:
%%cql create keyspace if not exists stock with replication = {'class':'SimpleStrategy','replication_factor':1}

In [ ]:
%%cql create table if not exists stock.trades_by_tickerday(
  day text,
  trade_id int,
  stock_symbol text,
  exchange text,
  trade_timestamp text,
  price float,
  quantity int,
  PRIMARY KEY ((stock_symbol,day), trade_timestamp, trade_id)
)

In [ ]:
trades.saveToCassandra("stock","trades_by_tickerday", SomeColumns("day","trade_id",
"stock_symbol","exchange","trade_timestamp","price","quantity"))

### Dataframe to Parquet Example

In [ ]:
import sqlContext.implicits._

Dataframe Example to Parquet

In [ ]:
val tradesDF = trades.toDF
tradesDF.printSchema

#### This will write it to CFS as parquet files

In [ ]:
tradesDF.write.parquet("/trades.parquet")